In [81]:
"""
李宏毅深度学习实验一 
线性回归
"""

'\n李宏毅深度学习实验一 \n线性回归\n'


#李宏毅深度学习实验一 
线性回归


In [82]:
import torch
import numpy as np
import pandas as pd 
import torch.nn as nn
import torch.optim
import torch.nn.parameter

In [83]:
#import data
data = pd.read_csv("./data/train.csv")

#每18个 一天 之后是0-9天的 数据
# train_data = data["Date","observation","0","1","2","3","4"]
# train_data.head()
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

In [84]:
data.head()
#将数据转换过去 换成正常的形态
month_data = {}
for month in range(12):
    sample = np.empty([18,480])
    for day in range(20):
        sample[:,day*24 :(day+1)*24] = raw_data[18*(20*month +day):18*(20*month+day+1),:]
    month_data[month] = sample


In [85]:
#每个月 480 个小时 9小时一个data 
#故471*12 笔
x = np.empty([12*471,18*9],dtype= float)
y = np.empty([12*471,1],dtype= float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day ==19 and hour >14:
                continue
            #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            x[month*471+day*24+hour,:] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            y[month*471+day*24+hour,0] = month_data[month][9, day * 24 + hour + 9]
print(x[0].size)#162 18*9
# print(y[0])


162


In [86]:
#归一化 可有可无
mean_x = np.mean(x,axis=0)
std_x = np.std(x,axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j]!=0:
            x[i][j] = (x[i][j]-mean_x[j])/std_x[j]

In [87]:

len(x[0]) #5652 *162
len(y[0])# 5652 *1

1

In [118]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.fc = nn.Linear(162,1)
        
    
    def forward(self,x):
        out = self.fc(x)
        return out

model = Model()

#setting
learing_rate = 0.05
epochs = 1000
optimizer = torch.optim.RMSprop(model.parameters(),lr=learing_rate)
# optimizer = torch.optim.Adagrad(torch.nn.parameter(),lr=0.01)
criterion = nn.MSELoss()

In [119]:
#train
# x = torch.from_numpy(x).float()
# y = torch.from_numpy(y).float()
for epoch in range(epochs):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(epoch%100==0):
        print("Epoch{}: {}".format(epoch,loss.item()))
torch.save(model,'model.pkl')


Epoch0: 737.1849975585938
Epoch100: 510.9842224121094
Epoch200: 485.3968200683594
Epoch300: 468.2388000488281
Epoch400: 454.74481201171875
Epoch500: 443.40728759765625
Epoch600: 433.52972412109375
Epoch700: 424.72125244140625
Epoch800: 416.7366943359375
Epoch900: 409.4101257324219


In [92]:
#test 数据处理
test_data = pd.read_csv('./data/test.csv', encoding='big5', header=None)
test_data = test_data.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_raw_data = test_data.to_numpy().astype(float)
test_tensor = torch.from_numpy(test_raw_data)

test_x = torch.zeros(240, 18*9)
test_split = torch.split(test_tensor, 18, dim=0)
print(len(test_split))
for i in range(len(test_split)):
    test_x[i] = torch.reshape(torch.t(test_split[i]), [1, -1])    # 注意此处要转置，因为处理训练集的时候转置了，而且转置之后才是比较和逻辑的。
# print(test_x[0])
norm = torch.nn.BatchNorm1d(162, affine=False, momentum=0.)      # 标准化
test_x = norm(test_x)

test_x2 = torch.cat([torch.ones(len(test_x), 1), test_x], dim=1)
# print(x)
print(test_x2.shape)
print(test_x.shape)


240
torch.Size([240, 163])
torch.Size([240, 162])


In [114]:
#test y 

testepoch = 1000
model2 = torch.load('model.pkl')
# test_x = torch.from_numpy(test_x)
test_y = model2(test_x)
# print(test_y.size)
print(len(test_y))
print(test_y.shape)
test_y = test_y.detach().numpy()
print(test_y[3][0])

240
torch.Size([240, 1])
21.361267


In [111]:
#save output.csv

import csv
with open("./data/submit.csv",mode = 'w',newline='')as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id','value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_'+ str(i),test_y[i][0]]
        csv_writer.writerow(row)
        # print(row)

['id', 'value']
